# Inferência

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Sempre que as pessoas comparam a inferência Bayesiana com as abordagens convencionais, uma das questões que surge com mais frequência é algo como: "E os valores p?

E um dos exemplos mais comuns é a comparação de dois grupos para ver se existe uma diferença nos seus meios.

Na inferência estatística clássica, a ferramenta habitual para este cenário é um [Student's *t*-test](https://en.wikipedia.org/wiki/Student%27s_t-test), e o resultado é um [p-value](https://en.wikipedia.org/wiki/P-value).

Este processo é um exemplo de [null hypothesis significance testing](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing).

Uma alternativa Bayesiana consiste em calcular a distribuição posterior da diferença entre os grupos.

Depois podemos utilizar essa distribuição para responder a quaisquer perguntas que nos interessem, incluindo o tamanho mais provável da diferença, um intervalo credível que é susceptível de conter a diferença verdadeira, a probabilidade de superioridade, ou a probabilidade de a diferença exceder algum limiar.

Para demonstrar este processo, resolverei um problema emprestado de um manual de estatística: avaliar o efeito de um "tratamento" educativo em comparação com um controlo.

## Melhorar a capacidade de leitura

Vamos utilizar dados de um [Ph.D. dissertation in educational psychology](https://docs.lib.purdue.edu/dissertations/AAI8807671/) escrito em 1987, que foi utilizado como exemplo num [statistics textbook](https://books.google.com/books/about/Introduction_to_the_practice_of_statisti.html?id=pGBNhajABlUC) de 1989 e publicado em [DASL](https://web.archive.org/web/20000603124754/http://lib.stat.cmu.edu/DASL/Datafiles/DRPScores.html), uma página web que recolhe histórias de dados.  

Aqui está a descrição do DASL:

> Um educador conduziu uma experiência para testar se novas actividades de leitura dirigida na sala de aula ajudarão os alunos do ensino básico a melhorar alguns aspectos da sua capacidade de leitura. Organizou uma turma da terceira classe de 21 alunos para acompanhar estas actividades durante um período de 8 semanas. Uma turma de controlo de 23 alunos da terceira classe seguiu o mesmo currículo sem as actividades. No final das 8 semanas, todos os alunos fizeram um teste de Grau de Poder de Leitura (DRP), que mede os aspectos da capacidade de leitura que o tratamento é concebido para melhorar.

O [dataset is available here](https://web.archive.org/web/20000603124754/http://lib.stat.cmu.edu/DASL/Datafiles/DRPScores.html).

A célula seguinte descarrega os dados.

In [4]:
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/drp_scores.csv')

Vou utilizar Pandas para carregar os dados para um "DataFrame".

In [5]:
import pandas as pd

df = pd.read_csv('drp_scores.csv', skiprows=21, delimiter='\t')
df.head(3)

A coluna "Tratamento" indica se cada aluno estava no grupo tratado ou no grupo de controlo.

A `Resposta` é a sua pontuação no teste.

Vou utilizar o "groupby" para separar os dados para os grupos "Tratado" e "Controlo":

In [6]:
grouped = df.groupby('Treatment')
responses = {}

for name, group in grouped:
    responses[name] = group['Response']

Aqui estão os CDFs das pontuações dos dois grupos e as estatísticas sumárias.

In [7]:
from empiricaldist import Cdf
from utils import decorate

for name, response in responses.items():
    cdf = Cdf.from_seq(response)
    cdf.plot(label=name)
    
decorate(xlabel='Score', 
         ylabel='CDF',
         title='Distributions of test scores')

Há sobreposição entre as distribuições, mas parece que as pontuações são mais elevadas no grupo tratado.

A distribuição das pontuações não é exactamente normal para nenhum dos grupos, mas é suficientemente próxima para que o modelo normal seja uma escolha razoável.

Assim, suponho que em toda a população de estudantes (não apenas nos da experiência), a distribuição das pontuações é bem modelada por uma distribuição normal com média e desvio padrão desconhecidos.

Vou utilizar `mu` e `sigma` para denotar estes parâmetros desconhecidos,

e faremos uma actualização Bayesiana para estimar o que eles são.

## Parâmetros de estimativa

Como sempre, precisamos de uma distribuição prévia para os parâmetros.

Uma vez que existem dois parâmetros, será uma distribuição conjunta.

Vou construí-lo escolhendo distribuições marginais para cada parâmetro e calculando o seu produto exterior.

Como simples ponto de partida, assumirei que as distribuições anteriores de "mu" e "sigma" são uniformes.

A função seguinte faz um objecto `Pmf` que representa uma distribuição uniforme.

In [8]:
from empiricaldist import Pmf

def make_uniform(qs, name=None, **options):
    """Make a Pmf that represents a uniform distribution."""
    pmf = Pmf(1.0, qs, **options)
    pmf.normalize()
    if name:
        pmf.index.name = name
    return pmf

`make_uniform` toma como parâmetros 

* Um conjunto de quantidades, `qs`, e

* Uma cadeia, `nome`, que é atribuída ao índice para que apareça quando exibimos o `Pmf`.

Aqui está a distribuição prévia para a `mu`:

In [9]:
import numpy as np

qs = np.linspace(20, 80, num=101)
prior_mu = make_uniform(qs, name='mean')

Eu escolhi os limites inferior e superior por tentativa e erro.

Explicarei como quando olharmos para a distribuição posterior.

Aqui está a distribuição prévia do `sigma`:

In [10]:
qs = np.linspace(5, 30, num=101)
prior_sigma = make_uniform(qs, name='std')

Agora podemos utilizar o `make_joint` para fazer a distribuição prévia conjunta.

In [11]:
from utils import make_joint

prior = make_joint(prior_mu, prior_sigma)

E vamos começar por trabalhar com os dados do grupo de controlo.

In [12]:
data = responses['Control']
data.shape

Na secção seguinte vamos calcular a probabilidade destes dados para cada par de parâmetros na distribuição anterior.

## Probabilidade

Gostaríamos de saber a probabilidade de cada pontuação no conjunto de dados para cada par hipotético de valores, `mu` e `sigma`.

Farei isso fazendo uma grelha tridimensional com valores de `mu` no primeiro eixo, valores de `sigma` no segundo eixo, e as pontuações do conjunto de dados no terceiro eixo.

In [13]:
mu_mesh, sigma_mesh, data_mesh = np.meshgrid(
    prior.columns, prior.index, data)

mu_mesh.shape

Agora podemos utilizar o `norm.pdf` para calcular a densidade de probabilidade de cada pontuação para cada par hipotético de parâmetros.

In [14]:
from scipy.stats import norm

densities = norm(mu_mesh, sigma_mesh).pdf(data_mesh)
densities.shape

O resultado é um conjunto 3-D.  Para calcular as probabilidades, multiplicarei estas densidades ao longo de `eixo=2`, que é o eixo dos dados:

In [15]:
likelihood = densities.prod(axis=2)
likelihood.shape

O resultado é um conjunto 2-D que contém a probabilidade de todo o conjunto de dados para cada par hipotético de parâmetros.

Podemos utilizar esta matriz para actualizar o anterior, desta forma:

In [16]:
from utils import normalize

posterior = prior * likelihood
normalize(posterior)
posterior.shape

O resultado é um "DataFrame" que representa a distribuição posterior conjunta.

A função que se segue encapsula estas etapas.

In [17]:
def update_norm(prior, data):
    """Update the prior based on data."""
    mu_mesh, sigma_mesh, data_mesh = np.meshgrid(
        prior.columns, prior.index, data)
    
    densities = norm(mu_mesh, sigma_mesh).pdf(data_mesh)
    likelihood = densities.prod(axis=2)
    
    posterior = prior * likelihood
    normalize(posterior)

    return posterior

Aqui estão as actualizações para os grupos de controlo e tratamento:

In [18]:
data = responses['Control']
posterior_control = update_norm(prior, data)

In [19]:
data = responses['Treated']
posterior_treated = update_norm(prior, data)

E aqui está como eles são:

In [20]:
import matplotlib.pyplot as plt
from utils import plot_contour

plot_contour(posterior_control, cmap='Blues')
plt.text(49.5, 18, 'Control', color='C0')

cs = plot_contour(posterior_treated, cmap='Oranges')
plt.text(57, 12, 'Treated', color='C1')

decorate(xlabel='Mean (mu)', 
         ylabel='Standard deviation (sigma)',
         title='Joint posterior distributions of mu and sigma')

Ao longo do eixo $x$, parece que a pontuação média para o grupo tratado é mais alta.

Ao longo do eixo de $y$, parece que o desvio padrão para o grupo tratado é mais baixo.

Se pensarmos que o tratamento causa estas diferenças, os dados sugerem que o tratamento aumenta a média das pontuações e diminui a sua dispersão.

Podemos ver estas diferenças com mais clareza analisando as distribuições marginais para `mu` e `sigma`.

## Distribuições Marginais Posteriores

Vou utilizar `marginal`, que vimos em <<_MarginalDistribuições>>, para extrair as distribuições marginais posteriores para os meios da população.

In [21]:
from utils import marginal

pmf_mean_control = marginal(posterior_control, 0)
pmf_mean_treated = marginal(posterior_treated, 0)

Eis como eles são:

In [22]:
pmf_mean_control.plot(label='Control')
pmf_mean_treated.plot(label='Treated')

decorate(xlabel='Population mean (mu)', 
         ylabel='PDF', 
         title='Posterior distributions of mu')

Em ambos os casos, as probabilidades posteriores nos extremos do intervalo são próximas de zero, o que significa que os limites que escolhemos para a distribuição prévia são suficientemente amplos.

Comparando as distribuições marginais para os dois grupos, parece que a média da população no grupo tratado é mais elevada.

Podemos utilizar o `prob_gt` para calcular a probabilidade de superioridade:

In [23]:
Pmf.prob_gt(pmf_mean_treated, pmf_mean_control)

Existe uma probabilidade de 98% de que a média no grupo tratado seja mais elevada.

## Distribuição das diferenças

Para quantificar a magnitude da diferença entre grupos, podemos utilizar o `sub_dist` para calcular a distribuição da diferença.

In [24]:
pmf_diff = Pmf.sub_dist(pmf_mean_treated, pmf_mean_control)

Há duas coisas com que ter cuidado quando se utilizam métodos como o `sub_dist`. 

O primeiro é que o resultado geralmente contém mais elementos do que o `Pmf` original.  

Neste exemplo, as distribuições originais têm as mesmas quantidades, pelo que o aumento do tamanho é moderado.

In [25]:
len(pmf_mean_treated), len(pmf_mean_control), len(pmf_diff)

Na pior das hipóteses, o tamanho do resultado pode ser o produto dos tamanhos dos originais.

A outra coisa a ter cuidado é traçar o `Pmf`.

Neste exemplo, se traçarmos a distribuição das diferenças, o resultado é bastante ruidoso.

In [26]:
pmf_diff.plot()

decorate(xlabel='Difference in population means', 
         ylabel='PDF', 
         title='Posterior distribution of difference in mu')

Há duas maneiras de contornar essa limitação.  Uma é traçar o CDF, que suaviza o ruído:

In [27]:
cdf_diff = pmf_diff.make_cdf()

In [28]:
cdf_diff.plot()

decorate(xlabel='Difference in population means', 
         ylabel='CDF', 
         title='Posterior distribution of difference in mu')

A outra opção é utilizar a estimativa da densidade do núcleo (KDE) para fazer uma aproximação suave do PDF numa grelha igualmente espaçada, que é o que esta função faz:

In [29]:
from scipy.stats import gaussian_kde

def kde_from_pmf(pmf, n=101):
    """Make a kernel density estimate for a PMF."""
    kde = gaussian_kde(pmf.qs, weights=pmf.ps)
    qs = np.linspace(pmf.qs.min(), pmf.qs.max(), n)
    ps = kde.evaluate(qs)
    pmf = Pmf(ps, qs)
    pmf.normalize()
    return pmf

O `kde_from_pmf` toma como parâmetros um `Pmf` e o número de lugares para avaliar o KDE.

Utiliza `gaussian_kde`, que vimos em <<_KernelDensityEstimation>>, passando as probabilidades do `Pmf` como pesos.

Isto torna as densidades estimadas mais elevadas onde as probabilidades no `Pmf` são mais elevadas.

Eis como se apresenta a estimativa da densidade do grão para o `Pmf` das diferenças entre os grupos.

In [30]:
kde_diff = kde_from_pmf(pmf_diff)

In [31]:
kde_diff.plot()

decorate(xlabel='Difference in means', 
         ylabel='PDF', 
         title='Posterior distribution of difference in mu')

A média desta distribuição é de quase 10 pontos num teste em que a média ronda os 45, pelo que o efeito do tratamento parece ser substancial.

In [32]:
pmf_diff.mean()

Podemos utilizar o `intervalo_crível` para calcular um intervalo credível de 90%.

In [33]:
pmf_diff.credible_interval(0.9)

Com base neste intervalo, estamos bastante seguros de que o tratamento melhora as pontuações dos testes em 2 a 17 pontos.

## Usando Estatísticas Sumárias

Neste exemplo o conjunto de dados não é muito grande, pelo que não demora muito tempo a calcular a probabilidade de cada pontuação sob cada hipótese.

Mas o resultado é um conjunto 3-D; para conjuntos de dados maiores, pode ser demasiado grande para calcular na prática.

Além disso, com conjuntos de dados maiores, as probabilidades ficam muito pequenas, por vezes tão pequenas que não as podemos calcular com aritmética de ponto flutuante.

Isto porque estamos a calcular a probabilidade de um determinado conjunto de dados; o número de conjuntos de dados possíveis é astronomicamente grande, por isso a probabilidade de qualquer um deles ser muito pequena.

Uma alternativa é calcular um resumo do conjunto de dados e calcular a probabilidade do resumo.

Por exemplo, se calcularmos a média e o desvio padrão dos dados, podemos calcular a probabilidade dessas estatísticas sumárias sob cada hipótese.

Como exemplo, suponhamos que sabemos que a média real da população, $\mu$, é 42 e o desvio padrão real, $\sigma$, é 17.

In [34]:
mu = 42
sigma = 17

Agora suponhamos que retiramos uma amostra desta distribuição com tamanho de amostra `n=20`, e calculamos a média da amostra, que chamarei `m`, e o desvio padrão da amostra, que chamarei `s`.

E suponhamos que se descobre que:

In [35]:
n = 20
m = 41
s = 18

As estatísticas sumárias, `m` e `s`, não estão muito longe dos parâmetros $\mu$ e $\sigma$, por isso parece que não são muito improváveis.

Para calcular a sua probabilidade, iremos tirar partido de três resultados de estatísticas matemáticas:

* Dado $\mu$ e $\sigma$, a distribuição de `m` é normal com parâmetros $\u$ e $\sigma/\sqrt{n$;

* A distribuição de $s$ é mais complicada, mas se calcularmos a transformação $t = n s^2 / \sigma^2$, a distribuição de $t$ é qui-quadrada com o parâmetro $n-1$; e

* De acordo com [Basu's theorem](https://en.wikipedia.org/wiki/Basu%27s_theorem), `m` e `s` são independentes.

Portanto, vamos calcular a probabilidade de "m" e "s" dados $\mu$ e $\sigma$.

Primeiro vou criar um objecto `norm` que representa a distribuição de `m`.

In [36]:
dist_m = norm(mu, sigma/np.sqrt(n))

Esta é a "distribuição de amostras da média".

Podemos utilizá-lo para calcular a probabilidade do valor observado de `m`, que é 41.

In [37]:
like1 = dist_m.pdf(m)
like1

Agora vamos calcular a probabilidade do valor observado de `s', que é 18.

Primeiro, calculamos o valor transformado `t`:

In [38]:
t = n * s**2 / sigma**2
t

Depois criamos um objecto `chi2` para representar a distribuição do `t`:

In [39]:
from scipy.stats import chi2

dist_s = chi2(n-1)

Agora podemos calcular a probabilidade de "não":

In [40]:
like2 = dist_s.pdf(t)
like2

Finalmente, porque `m` e `s` são independentes, a sua probabilidade conjunta é o produto das suas probabilidades:

In [41]:
like = like1 * like2
like

Agora podemos calcular a probabilidade dos dados para quaisquer valores de $\mu$ e $\sigma$, que utilizaremos na próxima secção para fazer a actualização.

## Actualização com Estatísticas Sumárias

Agora estamos prontos para fazer uma actualização.

Vou calcular as estatísticas sumárias para os dois grupos.

In [42]:
summary = {}

for name, response in responses.items():
    summary[name] = len(response), response.mean(), response.std()
    
summary

O resultado é um dicionário que mapeia desde o nome do grupo até um tuple que contém o tamanho da amostra, `n', a média da amostra, `m', e o desvio padrão da amostra `s', para cada grupo.

Vou demonstrar a actualização com o resumo estatístico do grupo de controlo.

In [43]:
n, m, s = summary['Control']

Vou fazer uma malha com valores hipotéticos de `mu` no eixo `x` e valores de `sigma` no eixo `y`.

In [44]:
mus, sigmas = np.meshgrid(prior.columns, prior.index)
mus.shape

Agora podemos calcular a probabilidade de ver a média da amostra, `m`, para cada par de parâmetros.

In [45]:
like1 = norm(mus, sigmas/np.sqrt(n)).pdf(m)
like1.shape

E podemos calcular a probabilidade do desvio padrão da amostra, `s', para cada par de parâmetros.

In [46]:
ts = n * s**2 / sigmas**2
like2 = chi2(n-1).pdf(ts)
like2.shape

Finalmente, podemos fazer a actualização com ambas as probabilidades:

In [47]:
posterior_control2 = prior * like1 * like2
normalize(posterior_control2)

Para calcular a distribuição posterior para o grupo de tratamento, vou colocar as etapas anteriores numa função:

In [48]:
def update_norm_summary(prior, data):
    """Update a normal distribution using summary statistics."""
    n, m, s = data
    mu_mesh, sigma_mesh = np.meshgrid(prior.columns, prior.index)
    
    like1 = norm(mu_mesh, sigma_mesh/np.sqrt(n)).pdf(m)
    like2 = chi2(n-1).pdf(n * s**2 / sigma_mesh**2)
    
    posterior = prior * like1 * like2
    normalize(posterior)
    
    return posterior

Aqui está a actualização para o grupo de tratamento:

In [49]:
data = summary['Treated']
posterior_treated2 = update_norm_summary(prior, data)

E aqui estão os resultados.

In [50]:
plot_contour(posterior_control2, cmap='Blues')
plt.text(49.5, 18, 'Control', color='C0')

cs = plot_contour(posterior_treated2, cmap='Oranges')
plt.text(57, 12, 'Treated', color='C1')

decorate(xlabel='Mean (mu)', 
         ylabel='Standard deviation (sigma)',
         title='Joint posterior distributions of mu and sigma')

Visualmente, estas distribuições conjuntas posteriores são semelhantes às que calculámos utilizando todo o conjunto de dados, e não apenas o resumo estatístico.

Mas não são exactamente as mesmas, como podemos ver ao comparar as distribuições marginais.

## Comparação de Marginais

Mais uma vez, vamos extrair as distribuições marginais posteriores.

In [51]:
from utils import marginal

pmf_mean_control2 = marginal(posterior_control2, 0)
pmf_mean_treated2 = marginal(posterior_treated2, 0)

E compará-los com os resultados obtidos utilizando todo o conjunto de dados (as linhas tracejadas).

In [52]:
pmf_mean_control.plot(color='C5', linestyle='dashed')
pmf_mean_control2.plot(label='Control')
pmf_mean_treated.plot(color='C5', linestyle='dashed')
pmf_mean_treated2.plot(label='Treated')

decorate(xlabel='Population mean', 
         ylabel='PDF', 
         title='Posterior distributions of mu')

As distribuições posteriores baseadas em estatísticas sumárias são semelhantes aos posteriors que calculámos utilizando todo o conjunto de dados, mas em ambos os casos são mais curtas e um pouco mais largas.

Isto porque a actualização com estatísticas sumárias se baseia na hipótese implícita de que a distribuição dos dados é normal.

Mas não é; como resultado, quando substituímos o conjunto de dados por estatísticas sumárias, perdemos alguma informação sobre a verdadeira distribuição dos dados.

Com menos informação, estamos menos seguros sobre os parâmetros.

## Comprovação por simulação

A actualização com estatísticas sumárias baseia-se em distribuições teóricas, e parece funcionar, mas penso que é útil testar teorias como esta, por algumas razões:

* Confirma que a nossa compreensão da teoria é correcta,

* Confirma que as condições em que aplicamos a teoria são condições em que a teoria se mantém, 

* Confirma que os detalhes de implementação estão correctos.  Para muitas distribuições, há mais do que uma forma de especificar os parâmetros.  Se utilizar a especificação errada, este tipo de testes ajudá-lo-á a detectar o erro.

Nesta secção vou utilizar simulações para mostrar que a distribuição da média da amostra e do desvio padrão é como eu afirmei.

Mas se quiserem acreditar na minha palavra, podem saltar esta secção e a seguinte.

Vamos supor que conhecemos a média real e o desvio padrão da população:

In [53]:
mu = 42
sigma = 17

Vou criar um objecto "normal" para representar esta distribuição.

In [54]:
dist = norm(mu, sigma)

O `norm` fornece `rvs`, o que gera valores aleatórios a partir da distribuição.

Podemos utilizá-lo para simular 1000 amostras, cada uma com tamanho de amostra `n=20`.

In [55]:
n = 20
samples = dist.rvs((1000, n))
samples.shape

O resultado é um conjunto com 1000 filas, cada uma contendo uma amostra ou 20 resultados simulados de testes.

Se calcularmos a média de cada linha, o resultado é um conjunto que contém 1000 meios de amostra; ou seja, cada valor é a média de uma amostra com `n=20`.

In [56]:
sample_means = samples.mean(axis=1)
sample_means.shape

Agora, vamos comparar a distribuição destes meios com o `dist_m`.

Vou utilizar `pmf_from_dist` para fazer uma aproximação discreta do `dist_m`:

In [57]:
def pmf_from_dist(dist, low, high):
    """Make a discrete approximation of a continuous distribution.
    
    dist: SciPy dist object
    low: low end of range
    high: high end of range
    
    returns: normalized Pmf
    """
    qs = np.linspace(low, high, 101)
    ps = dist.pdf(qs)
    pmf = Pmf(ps, qs)
    pmf.normalize()
    return pmf

O `pmf_from_dist` toma um objecto que representa uma distribuição contínua, avalia a sua função de densidade de probabilidade em pontos igualmente espaciais entre o `baixo` e o `alto`, e devolve um `Pmf` normalizado que se aproxima da distribuição.

Vou utilizá-lo para avaliar o `dist_m` numa gama de seis desvios padrão.

In [58]:
low = dist_m.mean() - dist_m.std() * 3
high = dist_m.mean() + dist_m.std() * 3

pmf_m = pmf_from_dist(dist_m, low, high)

Agora vamos comparar esta distribuição teórica com os meios das amostras.

Vou utilizar o `kde_from_sample` para estimar a sua distribuição e avaliá-la nos mesmos locais que o `pmf_m`.

In [59]:
from utils import kde_from_sample

qs = pmf_m.qs
pmf_sample_means = kde_from_sample(sample_means, qs)

A figura seguinte mostra as duas distribuições.

In [60]:
pmf_m.plot(label='Theoretical distribution',
           style=':', color='C5')
pmf_sample_means.plot(label='KDE of sample means')

decorate(xlabel='Mean score',
         ylabel='PDF',
         title='Distribution of the mean')

A distribuição teórica e a distribuição de meios de amostra estão de acordo.

## Verificação do Desvio Padrão

Vamos também verificar se os desvios padrão seguem a distribuição que esperamos.

Primeiro vou calcular o desvio padrão para cada uma das 1000 amostras.



In [61]:
sample_stds = samples.std(axis=1)
sample_stds.shape

Agora vamos calcular os valores transformados, $t = n s^2 / \sigma^2$.

In [62]:
transformed = n * sample_stds**2 / sigma**2

Esperamos que os valores transformados sigam uma distribuição qui-quadrada com o parâmetro $n-1$.

SciPy fornece `chi2`, que podemos utilizar para representar esta distribuição.

In [63]:
from scipy.stats import chi2

dist_s = chi2(n-1)

Podemos utilizar novamente `pmf_from_dist` para fazer uma aproximação discreta.

In [64]:
low = 0
high = dist_s.mean() + dist_s.std() * 4

pmf_s = pmf_from_dist(dist_s, low, high)

E utilizaremos `kde_from_sample` para estimar a distribuição dos desvios padrão da amostra.

In [65]:
qs = pmf_s.qs
pmf_sample_stds = kde_from_sample(transformed, qs)

Agora podemos comparar a distribuição teórica com a distribuição dos desvios padrão.

In [66]:
pmf_s.plot(label='Theoretical distribution',
           style=':', color='C5')
pmf_sample_stds.plot(label='KDE of sample std',
                     color='C1')

decorate(xlabel='Standard deviation of scores',
         ylabel='PDF',
         title='Distribution of standard deviation')

A distribuição de desvios padrão transformados está de acordo com a distribuição teórica.

Finalmente, para confirmar que os meios da amostra e os desvios padrão são independentes, vou calcular o seu coeficiente de correlação:

In [67]:
np.corrcoef(sample_means, sample_stds)[0][1]

A sua correlação é quase nula, o que é consistente com o facto de serem independentes.

Assim, as simulações confirmam os resultados teóricos que utilizámos para fazer a actualização com estatísticas sumárias.

Também podemos utilizar o `kdeplot` da Seaborn para ver como é a sua distribuição conjunta.

In [68]:
import seaborn as sns

sns.kdeplot(x=sample_means, y=sample_stds)

decorate(xlabel='Mean (mu)',
         ylabel='Standard deviation (sigma)',
         title='Joint distribution of mu and sigma')

Parece que os eixos das elipses estão alinhados com os eixos, o que indica que as variáveis são independentes.

## Resumo

Neste capítulo, utilizámos uma distribuição conjunta para representar probabilidades anteriores para os parâmetros de uma distribuição normal, `mu` e `sigma`.

E actualizámos essa distribuição de duas maneiras: primeiro utilizando todo o conjunto de dados e o PDF normal; depois utilizando estatísticas resumidas, o PDF normal, e o PDF qui-quadrado.

A utilização de estatísticas sumárias é computacionalmente mais eficiente, mas perde alguma informação no processo.

As distribuições normais aparecem em muitos domínios, pelo que os métodos deste capítulo são amplamente aplicáveis.  Os exercícios no final do capítulo dar-lhe-ão uma oportunidade de os aplicar.

## Exercícios

**Exercício:** Olhando novamente para a distribuição conjunta posterior de `mu` e `sigma`, parece que o desvio padrão do grupo tratado pode ser menor; se assim for, isso sugere que o tratamento é mais eficaz para estudantes com pontuações mais baixas.

Mas antes de especularmos demasiado, devemos estimar o tamanho da diferença e ver se ela pode realmente ser 0.

Extrair as distribuições marginais posteriores de `sigma` para os dois grupos.

Qual é a probabilidade de o desvio padrão ser maior no grupo de controlo?

Calcular a distribuição da diferença em `sigma` entre os dois grupos.  Qual é o significado desta diferença?  Qual é o intervalo credível de 90%?

In [69]:
# Solution goes here

In [70]:
# Solution goes here

In [71]:
# Solution goes here

In [72]:
# Solution goes here

In [73]:
# Solution goes here

In [74]:
# Solution goes here

In [75]:
# Solution goes here

**Exercício:** Uma [effect size](http://en.wikipedia.org/wiki/Effect_size) é uma estatística destinada a quantificar a magnitude de um fenómeno.

Se o fenómeno é uma diferença de meios entre dois grupos, uma forma comum de o quantificar é o tamanho do efeito de Cohen, denotado $d$.

Se os parâmetros do Grupo 1 forem $(\mu_1, \sigma_1)$, e o

Os parâmetros para o Grupo 2 são $(\mu_2, \sigma_2)$, Cohen's

tamanho do efeito é

$$ d = \frac{\mu_1 - \mu_2}{(\sigma_1 + \sigma_2)/2} $$

Utilizar as distribuições posteriores conjuntas para os dois grupos para calcular a distribuição posterior para o tamanho do efeito de Cohen.

Se tentarmos enumerar todos os pares das duas distribuições, é preciso também

longo, pelo que utilizaremos a amostragem aleatória.

A seguinte função toma uma distribuição posterior conjunta e devolve uma amostra de pares.

Utiliza algumas características que ainda não vimos, mas pode ignorar os detalhes por agora.

In [76]:
def sample_joint(joint, size):
    """Draw a sample from a joint distribution.
    
    joint: DataFrame representing a joint distribution
    size: sample size
    """
    pmf = Pmf(joint.transpose().stack())
    return pmf.choice(size)

Eis como podemos utilizá-lo para amostrar pares a partir das distribuições posteriores para os dois grupos.

In [77]:
sample_treated = sample_joint(posterior_treated, 1000)
sample_treated.shape

In [78]:
sample_control = sample_joint(posterior_control, 1000)
sample_control.shape

O resultado é um conjunto de tuplos, onde cada tuple contém um possível par de valores para $\mu$ e $\sigma$.

Agora é possível percorrer as amostras, calcular o tamanho do efeito Cohen para cada uma, e estimar a distribuição dos tamanhos do efeito.

In [79]:
# Solution goes here

In [80]:
# Solution goes here

In [81]:
# Solution goes here

In [82]:
# Solution goes here

In [83]:
# Solution goes here

In [84]:
# Solution goes here

**Exercício:** Este exercício é inspirado por [a question that appeared on Reddit](https://www.reddit.com/r/statistics/comments/hcvl2j/q_reverse_empirical_distribution_rule_question/).

Um instrutor anuncia os resultados de um exame como este: "A nota média neste exame foi 81.  De 25 estudantes, 5 obtiveram mais de 90, e tenho o prazer de informar que ninguém reprovou (obtiveram menos de 60)".

Com base nestas informações, qual foi, na sua opinião, o desvio padrão das pontuações?

Pode assumir que a distribuição das pontuações é aproximadamente normal.  E vamos supor que a média da amostra, 81, é na realidade a média da população, por isso só temos de estimar o `sigma`.

Dica: Para calcular a probabilidade de uma pontuação superior a 90, pode utilizar `norm.sf`, que calcula a função de sobrevivência, também conhecida como CDF complementar, ou `1 - cdf(x)`.

In [85]:
# Solution goes here

In [86]:
# Solution goes here

In [87]:
# Solution goes here

In [88]:
# Solution goes here

In [89]:
# Solution goes here

In [90]:
# Solution goes here

In [91]:
# Solution goes here

In [92]:
# Solution goes here

In [93]:
# Solution goes here

In [94]:
# Solution goes here

In [95]:
# Solution goes here

In [96]:
# Solution goes here

**Exercício:** A [Variability Hypothesis](http://en.wikipedia.org/wiki/Variability_hypothesis) é a observação de que muitos traços físicos são mais variáveis entre os machos do que entre as fêmeas, em muitas espécies. 

Tem sido um tema de controvérsia desde o início do século XIX, o que sugere um exercício que podemos utilizar para praticar os métodos deste capítulo.  Vejamos a distribuição das alturas para homens e mulheres nos Estados Unidos e vejamos quem é mais variável.

Utilizei os dados de 2018 do CDC [Behavioral Risk Factor Surveillance System](https://www.cdc.gov/brfss/annual_data/annual_2018.html) (BRFSS), que inclui alturas declaradas por 154 407 homens e 254 722 mulheres. 

Aqui está o que encontrei:

* A altura média para os homens é de 178 cm; a altura média para as mulheres é de 163 cm. Portanto, os homens são mais altos em média; não é de admirar que assim seja.

* Para os homens o desvio padrão é de 8,27 cm; para as mulheres é de 7,75 cm. Assim, em termos absolutos, as alturas dos homens são mais variáveis.

Mas para comparar a variabilidade entre grupos, é mais significativo utilizar o [coefficient of variation](https://en.wikipedia.org/wiki/Coefficient_of_variation) (CV), que é o desvio padrão dividido pela média. É uma medida sem dimensão da variabilidade em relação à escala. 

Para homens o CV é de 0,0465; para mulheres é de 0,0475.

O coeficiente de variação é mais elevado para as mulheres, pelo que este conjunto de dados fornece provas contra a Hipótese de Variabilidade. Mas podemos utilizar métodos Bayesianos para tornar essa conclusão mais precisa.

Utilize estas estatísticas resumidas para calcular a distribuição posterior de `mu` e `sigma` para as distribuições de altura masculina e feminina.

Utilize `Pmf.div_dist` para calcular as distribuições posteriores de CV.

Com base neste conjunto de dados e na hipótese de que a distribuição da altura é normal, qual é a probabilidade de o coeficiente de variação ser mais elevado para os homens?

Qual é o rácio mais provável dos CV e qual é o intervalo credível de 90% para esse rácio?

Dica: Utilizar distribuições prévias diferentes para os dois grupos, e escolhê-las de modo a cobrir todos os parâmetros com probabilidades não negligenciáveis.

Além disso, poderá achar esta função útil:

In [97]:
def get_posterior_cv(joint):
    """Get the posterior distribution of CV.
    
    joint: joint distribution of mu and sigma
    
    returns: Pmf representing the smoothed posterior distribution
    """
    pmf_mu = marginal(joint, 0)
    pmf_sigma = marginal(joint, 1)
    pmf_cv = Pmf.div_dist(pmf_sigma, pmf_mu)
    return kde_from_pmf(pmf_cv)

In [98]:
# Solution goes here

In [99]:
# Solution goes here

In [100]:
# Solution goes here

In [101]:
# Solution goes here

In [102]:
# Solution goes here

In [103]:
# Solution goes here

In [104]:
# Solution goes here

In [105]:
# Solution goes here

In [106]:
# Solution goes here